In [1]:
# To capture the NNP library stdout output we have two possible options:
#
# 1.) Install "wurlitzer" available via pip or github.com/minrk/wurlitzer
#     > pip install wurlitzer
#     and load as extension
#     > %load_ext wurlitzer
#
# 2.) Silence the library output (which will otherwise go to Jupyter server shell)
#     > nnp = pynnp.Mode()
#     > nnp.log.writeToStdout = False
#     > ... do initialization ...
#     and recall the log later:
#     > for line in nnp.log.getLog():
#     >     sys.stdout.write(line)
#
# Option 1.) is obviously more comfortable and will also capture output to stderr.
%load_ext wurlitzer
import pynnp
from sys import stdout as so

In [2]:
# Create instance of NNP setup.                               
nnp = pynnp.Mode()

# Turn off output to stdout (for Option 2.) described above).
#nnp.log.writeToStdout = False

In [3]:
# Run initialization steps (symmetry functions only).
nnp.initialize()                                                                  
nnp.loadSettingsFile("input.nn")                                                  
nnp.setupElementMap()                                                             
nnp.setupElements()                                                               
nnp.setupCutoff()                                                                 
nnp.setupSymmetryFunctions()                                                      
nnp.setupSymmetryFunctionGroups()                                                 
nnp.setupSymmetryFunctionScaling("scaling.data")                                  
nnp.setupSymmetryFunctionStatistics(False, False, True, False)

# Print log file (for Option 2.) described above).
#for line in m.log.getLog():
#    so.write(line)


*******************************************************************************

   NNP LIBRARY v2.0.0
   ------------------

Git branch  : n2p2-release-v2
Git revision: 241f477 (241f4774efa595643a8cda93fedff4aa46d03bcd)

Number of OpenMP threads: 4
*******************************************************************************

*** SETUP: SETTINGS FILE ******************************************************

Settings file name: input.nn
Read 96 lines.
Found 60 lines with keywords.
*******************************************************************************

*** SETUP: ELEMENT MAP ********************************************************

Number of element strings found: 2
Element  0:  H (  1)
Element  1:  O (  8)
*******************************************************************************

*** SETUP: ELEMENTS ***********************************************************

Number of elements is consistent: 2
Atomic energy offsets per element:
Element  0:   0.00000000E+00
Element  1: 

In [4]:
# Create an empty structure.
struct = pynnp.Structure()

# Tell the structure which elements are to be expected,
# i.e. pass the ElementMap from the NNP setup instance.
# Note: this step is required!
struct.setElementMap(nnp.elementMap)

In [5]:
# Read in configuration from file.
# WARNING: Do not read repeatedly as this will pile up atoms!
struct.readFromFile("input.data.1")

# Print some information:
print "Number of atoms: ", struct.numAtoms
print "Number of atoms per element: "
for (i, n) in enumerate(struct.numAtomsPerElement):
    print nnp.elementMap[i], ": ", n

Number of atoms:  24
Number of atoms per element: 
H :  16
O :  8


In [6]:
# Retrieve cutoff radius form NNP setup.                                        
cutoffRadius = nnp.getMaxCutoffRadius()                                           
print "Cutoff radius = ", cutoffRadius                                          
                                                                                
# Calculate neighbor list.                                                      
struct.calculateNeighborList(cutoffRadius)

# Show some information about neighbor list.
for atom in struct.atoms:
    print "Atom {0:4d} ({1:2s}): {2:3d} neighbors".format(atom.index, nnp.elementMap[atom.element], atom.numNeighbors)

Cutoff radius =  12.0
Atom    0 (O ): 142 neighbors
Atom    1 (O ): 142 neighbors
Atom    2 (O ): 142 neighbors
Atom    3 (O ): 142 neighbors
Atom    4 (O ): 142 neighbors
Atom    5 (O ): 142 neighbors
Atom    6 (O ): 142 neighbors
Atom    7 (O ): 142 neighbors
Atom    8 (H ): 127 neighbors
Atom    9 (H ): 127 neighbors
Atom   10 (H ): 127 neighbors
Atom   11 (H ): 127 neighbors
Atom   12 (H ): 127 neighbors
Atom   13 (H ): 127 neighbors
Atom   14 (H ): 127 neighbors
Atom   15 (H ): 128 neighbors
Atom   16 (H ): 127 neighbors
Atom   17 (H ): 127 neighbors
Atom   18 (H ): 128 neighbors
Atom   19 (H ): 127 neighbors
Atom   20 (H ): 127 neighbors
Atom   21 (H ): 127 neighbors
Atom   22 (H ): 127 neighbors
Atom   23 (H ): 127 neighbors


In [7]:
# Calculate symmetry functions for all atoms (using groups is generally faster).
#nnp.calculateSymmetryFunctions(s, False)
nnp.calculateSymmetryFunctionGroups(struct, False)
                                                                                
# Retrieve symmetry functions of atom with index 0.
# Symmetry functions sort order is provided in the library output
# of the setupSymmetryFunctions() method (see above).
print struct.atoms[0].G

[3.0979046907343974, 1.18994287347951, 2.3374571003048064, 0.8149627350606994, 1.4183860085998368, 0.365071888883069, 0.8566908159542491, 0.11517221925856677, 0.6816891712438031, 0.5470457335903648, 0.5140233759056304, 0.2789086480393483, 0.3795853440912408, 0.08424583073547427, 0.16736514762008486, 0.0037220328749432313, 0.017450677340565067, 0.0005024982243135199, 0.2321654300938664, 0.015056415734351522, 0.02580794766232825, 0.08085925995465679, 0.04351493344073904, 0.00607693892348457, 0.0003358516494455779, 0.048211127016197244, 0.029932837634662226, 0.000989359235199765, 0.01398246084208952, 0.010175918586600824]


In [11]:
# Repeat the procedure for another structure.

# First reset the structure, this will erase all data except for the ElementMap.
struct.reset()

struct.readFromFile("input.data.2")
struct.calculateNeighborList(cutoffRadius)
%time nnp.calculateSymmetryFunctionGroups(struct, False)
print struct.atoms[0].G

CPU times: user 45.1 ms, sys: 0 ns, total: 45.1 ms
Wall time: 21.5 ms
[2.86649377277164, 1.6229466695415529, 2.0623976954260583, 1.2201970615555326, 1.0990912879196195, 0.7414354094663689, 0.5115930320848044, 0.45659991006210426, 0.3768515741861182, 0.5650265555940194, 0.278675620398098, 0.2635626900178468, 0.19573039002565443, 0.12387824509705335, 0.0866044486663157, 0.03509025803430324, 0.02230215047551002, 0.031993442362933104, 0.019395663617398876, 0.1728751811637228, 0.01879781449053553, 0.007054820572245642, 0.08605197074485345, 0.004730188232226057, 0.003192176127904365, 0.022823231470083947, 0.0017684369314133184]
